In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [3]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

In [5]:
class Joke(BaseModel):
    """Broma para contar al usuario."""

    setup: str = Field(description="pregunta para montar una broma")
    punchline: str = Field(description="respuesta para resolver la broma")

    # Puedes añadir lógica de validación personalizada fácilmente con Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("¡Pregunta mal formulada!")
        return field

In [10]:
parser = PydanticToolsParser(tools=[Joke])

In [11]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192").bind_tools([Joke])

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "Eres un asistente útil"), ("user", "{input}")]
)

In [18]:
parser = PydanticToolsParser(tools=[Joke],first_tool_only=True, method="json_mode")

In [19]:
chain = prompt | llm | parser
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Eres un asistente útil')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D0C9C60E80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D0C9C618A0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Joke', 'description': 'Broma para contar al usuario.', 'parameters': {'type': 'object', 'properties': {'setup': {'description': 'pregunta para montar una broma', 'type': 'string'}, 'punchline': {'description': 'respuesta para resolver la broma', 'type': 'string'}}, 'required': ['setup', 'punchline']}}}]})
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.Joke

In [20]:
resultado = chain.invoke({"input": "cuéntame un chiste"})
resultado

Joke(setup='¿Por qué la computadora fue al médico?', punchline='¡Tenía un virus!')